In [11]:
import math
import random
import time
import numpy as np

In [2]:
def american_to_percentage(odds):
    if odds >= 100:
        return 100 / (odds + 100)
    elif odds <= -100:
        return abs(odds) / (abs(odds) + 100)
    else:
        print("Invalid Number, cannot be between -100 and 100")
        return None
    
def percentage_to_american(probability):
    if probability == 0:
        return None
    elif probability > 0.5:
        return round((probability * 100) / (1 - probability) * -1)
    else:
        return round((100 / probability) - 100)
    
def calculate_winnings(odds, bet_amount):
    if odds > 0:
        winnings = (odds / 100) * bet_amount
    else:
        winnings = (100 / abs(odds)) * bet_amount
    
    return round(winnings, 2)

def parlay(odds_list):
    parlay_probability = 1
    for odds in odds_list:
        parlay_probability *= american_to_percentage(odds)
    return parlay_probability   
    

In [148]:
odds = -135
paid_odds = -110
win = 0
lost = 0
goal = 300

for _ in range(1_000_000):
    current = 100
    amount = 1
    bets = 0

    while current > 0 and current < goal and bets < 1_000_000:
        bets += 1
        won = False
        while not won:
            if current <= 0:
                break
            if current - amount < 0:
                amount = current
            current -= amount
            num = random.random()
            if num < american_to_percentage(odds):
                won = True
                current += calculate_winnings(paid_odds, amount) + amount
                amount = 10
            else:
                amount *= 1
            if current >= goal:
                break
    if current > 0:
        win += 1
    else:
        lost += 1


print("WON:", win)
print("LOST:", lost)

print("Chances of Suceeding:", win / 1_000_000)

WON: 884478
LOST: 115522
Chances of Suceeding: 0.884478
